In [9]:
search_for = 65
start_from = 0
threads = 1
thread = 0
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.014595508575439453
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 25
all cases: 24164921
type: [1, 1, 1, 1, 65] 65
cases of this type: 17850625
10000 4.049793688693874
20000 9.605181262054861
30000 10.06061314266273
40000 10.782545370727902
50000 9.934454542717974
60000 10.059542094757617
70000 10.322972186287185
80000 10.637721696252962
90000 9.990944291181394
100000 9.123441503573796
110000 9.968033348153293
120000 9.784616970558968
130000 7.9937209834538105
140000 2.182994122291704
150000 4.275985386197693
160000 10.22645434226101
170000 9.552055266860961
180000 9.753456373614865
190000 9.460885071983112
200000 9.693856912937342
210000 10.929620830067721
220000 9.78968306491928
230000 9.672832666523913
240000 9.755166862227137
250000 9.586748374699612
260000 10.302300476250847
270000 6.42387103954786
280000 4.5470337086034815
290000 4.099440170170247
300000 9.21377312834843
310000 9.189536066139937
32000

3120000 5.4248049365353985
3130000 5.161377097284967
3140000 6.328632924401124
3150000 5.2139018834792505
3160000 1.8449867244912583
3170000 2.1024087675928476
3180000 6.420983643610206
3190000 6.386682102873955
3200000 6.292522257909119
3210000 5.240134137939692
3220000 6.216927730015417
3230000 6.754483294707537
3240000 5.281262164171061
3250000 4.134804055120633
3260000 4.9084638417840605
3270000 6.24323007274021
3280000 6.505014005077058
3290000 6.257159348713715
3300000 6.3237722284029525
3310000 5.664173645179676
3320000 5.734020703637918
3330000 4.189808474960069
3340000 5.099577707322915
3350000 3.341801205726922
3360000 3.8392490498110057
3370000 7.040733590858446
3380000 6.625171626847048
3390000 6.064108198675056
3400000 5.138738485039837
3410000 6.494227154875914
3420000 6.475516817690512
3430000 3.0570016411368384
3440000 1.843990081273079
3450000 4.4832320242100225
3460000 6.173600630468177
3470000 5.086389413862208
3480000 5.691721249074373
3490000 6.519441776762101
3500

6260000 4.921898300464173
6270000 5.028041736553655
6280000 4.892884460532844
6290000 4.348753314315836
6300000 4.313328499721189
6310000 5.400714329448502
6320000 5.089560793747902
6330000 5.524279975818455
6340000 5.571486371988939
6350000 5.806570568604747
6360000 5.821643672264093
6370000 4.693284610052433
6380000 5.22774139350764
6390000 4.452293693211351
6400000 5.250467497817802
6410000 4.538272608601808
6420000 2.637079922264285
6430000 3.421653917482747
6440000 5.846878349054853
6450000 2.467329410133991
6460000 1.5213184873885446
6470000 3.7624492618199787
6480000 5.569015369951169
6490000 3.383015314518975
6500000 2.7806629254769684
6510000 4.088064038768166
6520000 5.018694289772637
6530000 5.397516823944469
6540000 5.6106310359816485
6550000 5.050784593640963
6560000 4.440289765624365
6570000 5.650936986986014
6580000 3.8937057896030747
6590000 5.106181112522006
6600000 6.617130174027046
6610000 6.029397300870068
6620000 5.3668878770299955
6630000 4.47925905322912
6640000 

9370000 1.1466630939160916
9380000 1.0143843130010366
9390000 1.1093883125664923
9400000 0.9591159145295289
9410000 1.137342311652422
9420000 0.9852042409610483
9430000 1.0475434391372669
9440000 1.067831929051578
9450000 0.8690981816485193
9460000 1.1260342108107142
9470000 1.1157986749247908
9480000 1.1318141918666296
9490000 1.1120191685909695
9500000 1.0710733150623242
9510000 0.8892137087761693
9520000 1.029384648298184
9530000 1.0997814521355826
9540000 1.1040224699205556
9550000 1.08796185634417
9560000 1.092098353017211
9570000 1.0964766743561891
9580000 1.1086306877059737
9590000 1.1152587545256218
9600000 1.1132582053996392
9610000 1.0992058358071048
9620000 5.286226562841833
9630000 5.501589146852023
9640000 4.910445937469019
9650000 5.881136137284756
9660000 4.862664007728245
9670000 3.883478812275204
9680000 4.919435264858902
9690000 4.477730541153126
9700000 5.212014970445441
9710000 5.521517268957337
9720000 4.463760055347541
9730000 4.3622865585337145
9740000 2.43557327

12390000 2.6065333354650138
12400000 3.09219497975262
12410000 3.4709599817223147
12420000 2.9169997480584846
12430000 3.5792451045981384
12440000 2.149697564435959
12450000 1.852347361319032
12460000 3.003308030261788
12470000 2.483045294683933
12480000 3.6310163147368497
12490000 2.4133915124263496
12500000 1.0669527377972405
12510000 2.0000836765804424
12520000 3.75739836198365
12530000 3.5136200731785894
12540000 3.7276503441842395
12550000 1.6325414246597223
12560000 1.71244850935934
12570000 3.808887309400393
12580000 3.641080756667786
12590000 3.3977345171597
12600000 3.88740830557349
12610000 3.7447409554405406
12620000 3.1287620675088763
12630000 2.6534571757383145
12640000 3.547535567191469
12650000 3.7725986779314677
12660000 4.043571573827571
12670000 3.8899422410348388
12680000 3.85097017004176
12690000 3.5925867284307214
12700000 3.4278873672250576
12710000 2.0663844586557154
12720000 2.1977612748280366
12730000 3.2468178070081866
12740000 1.917783660152515
12750000 2.386

15370000 2.1283267050887344
15380000 1.640093898570021
15390000 1.6555224399625923
15400000 1.746509910729561
15410000 1.933659899524927
15420000 0.8551958494133022
15430000 1.609908869749811
15440000 1.5336628426826397
15450000 1.5461935518401795
15460000 1.465212246563885
15470000 2.34710702176555
15480000 1.8676072663478387
15490000 1.3488307852999037
15500000 2.3078865393871664
15510000 1.2194877343195742
15520000 0.3856233345258236
15530000 0.7760210583984057
15540000 1.740737500240134
15550000 1.4382354491988618
15560000 2.6042618565974833
15570000 2.860461409735792
15580000 1.598505150672416
15590000 1.4965061376851796
15600000 1.5134740888283849
15610000 1.3534201654873887
15620000 0.7959448454192082
15630000 0.9993093589438995
15640000 2.2171461514696413
15650000 1.4914397614444295
15660000 1.7996328825745382
15670000 2.6616893600796
15680000 1.9745713466840389
15690000 0.8333483078253202
15700000 1.1602707455504735
15710000 1.9468882988779743
15720000 2.232632347327087
157300

18300000 0.31345704252884127
18310000 0.29436647497002943
18320000 0.45551044879359004
18330000 0.38332479325456087
18340000 0.5520293508852323
18350000 0.59430890770638
18360000 0.4935298416330549
18370000 0.49124276795302496
18380000 0.4914435020126104
18390000 0.45860863992934753
18400000 0.47651217901294757
18410000 0.48649222420849403
18420000 0.4709791813055873
18430000 0.4535447784571978
18440000 0.5298662321856022
18450000 0.647893977816648
18460000 0.702588853270769
18470000 0.6974486852132677
18480000 0.6377292866203256
18490000 0.8616845035012033
18500000 0.5679735301964879
18510000 0.4735663562344048
18520000 0.25645031975628274
18530000 0.277296470366021
18540000 0.2509028616405659
18550000 0.25342786786287363
18560000 0.2528532083063125
18570000 0.26020555281712615
18580000 0.3073873675472273
18590000 0.30685931686762963
18600000 0.4798419822727309
18610000 0.41718908784068287
18620000 0.44597802004949255
18630000 0.3518445325974955
18640000 0.45286092244250903
18650000 0

21200000 0.21785847801095246
21210000 0.3245898604484068
21220000 0.3008130599490073
21230000 0.37547831808684273
21240000 0.2601706523388293
21250000 0.17694377214333082
21260000 0.17950248331707713
21270000 0.12911595936738118
21280000 0.21229870110869412
21290000 0.23512624231288828
21300000 0.20776412141108513
21310000 0.18171065095901487
21320000 0.1588630325652361
21330000 0.14380088831817442
21340000 0.3285037949648632
21350000 0.18937349600744247
21360000 0.27830837916493417
21370000 0.18548394264543056
21380000 0.2127787960471114
21390000 0.19696699609087573
21400000 0.20372248353108435
21410000 0.22734086517602203
21420000 0.3751079596327212
best so far: 0
type: [1, 1, 1, 13, 5] 65
cases of this type: 1373125
21430000 0.25860224957827727
21440000 0.2072673186171055
21450000 0.2958326060877442
21460000 0.1959516116030481
21470000 0.21450377961800496
21480000 0.19359860365724565
21490000 0.3228812691985435
21500000 0.20497695009783906
21510000 0.1616162724668516
21520000 0.1637

23920000 0.010006245123922826
23930000 0.00953578757329782
23940000 0.015787709784203106
best so far: 0
type: [1, 1, 13, 5, 1] 65
cases of this type: 21125
23950000 0.014307886759387121
23960000 0.007345673130750656
best so far: 0
type: [1, 1, 65, 1, 1] 65
cases of this type: 4225
best so far: 0
type: [1, 5, 1, 1, 13] 65
cases of this type: 142805
23970000 0.008083199174039893
23980000 0.01205564368437396
23990000 0.01701972519803047
24000000 0.009825442955063449
24010000 0.009613362820267677
24020000 0.007198586111426353
24030000 0.006243612212412888
24040000 0.005203495407283307
24050000 0.00478706135571003
24060000 0.004259924119472504
24070000 0.00473445466585954
24080000 0.005335001337607701
24090000 0.004413417409472996
24100000 0.0038490602223343316
24110000 0.004642892010986805
best so far: 0
type: [1, 5, 1, 13, 1] 65
cases of this type: 10985
24120000 0.0019722008676396475
best so far: 0
type: [1, 5, 13, 1, 1] 65
cases of this type: 845
best so far: 0
type: [1, 13, 1, 1, 5] 65